``` docker compose --profile default -f docker-compose.yaml up -d```

In [ ]:
import json
import druidapi
import requests
import configparser

druid_host = "http://localhost:8888"

druid_headers = {'Content-Type': 'application/json'}

print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)
display = druid.display
sql_client = druid.sql
status_client = druid.status
druid_headers = {'Content-Type': 'application/json'}

config = configparser.ConfigParser()
config.read('auth.ini')

status_client.version

In [9]:
ingestion_spec = {
  "type": "kafka",
  "spec": {
    "dataSchema": {
      "dataSource": "adsb-json",
      "timestampSpec": {
        "column": "kafka.timestamp",
        "format": "millis",
        "missingValue": None
      },
      "dimensionsSpec": {
        "dimensions": [
          {
            "type": "string",
            "name": "hex_ident_val",
            "multiValueHandling": "SORTED_ARRAY",
            "createBitmapIndex": True
          },
          {
            "type": "string",
            "name": "squawk",
            "multiValueHandling": "SORTED_ARRAY",
            "createBitmapIndex": True
          },
          {
            "type": "string",
            "name": "client_id",
            "multiValueHandling": "SORTED_ARRAY",
            "createBitmapIndex": True
          },
          {
            "type": "double",
            "name": "client_lat",
            "multiValueHandling": "SORTED_ARRAY",
            "createBitmapIndex": False
          },
          {
            "type": "double",
            "name": "client_lon",
            "multiValueHandling": "SORTED_ARRAY",
            "createBitmapIndex": False
          },
          {
            "type": "double",
            "name": "client_alt",
            "multiValueHandling": "SORTED_ARRAY",
            "createBitmapIndex": False
          },
          {
            "type": "double",
            "name": "distance_to_aircraft",
            "multiValueHandling": "SORTED_ARRAY",
            "createBitmapIndex": False
          }
        ],
        "dimensionExclusions": [
          "__time",
          "altitude",
          "latitude",
          "count",
          "kafka.timestamp",
          "ground_speed",
          "track",
          "vertical_rate",
          "longitude"
        ],
        "includeAllDimensions": False,
        "useSchemaDiscovery": False
      },
      "metricsSpec": [
        {
          "type": "count",
          "name": "count"
        },
        {
          "type": "doubleMax",
          "name": "altitude",
          "fieldName": "altitude"
        },
        {
          "type": "doubleMax",
          "name": "latitude",
          "fieldName": "latitude"
        },
        {
          "type": "doubleMax",
          "name": "ground_speed",
          "fieldName": "ground_speed"
        },
        {
          "type": "doubleMax",
          "name": "track",
          "fieldName": "track"
        },
        {
          "type": "doubleMax",
          "name": "longitude",
          "fieldName": "longitude"
        },
        {
          "type": "doubleMax",
          "name": "vertical_rate",
          "fieldName": "vertical_rate"
        }
      ],
      "granularitySpec": {
        "type": "uniform",
        "segmentGranularity": "DAY",
        "queryGranularity": "SECOND",
        "rollup": True,
        "intervals": []
      },
      "transformSpec": {
        "filter": None,
        "transforms": [
          {
            "type": "expression",
            "name": "distance_to_aircraft",
            "expression": "(acos(((sin(toRadians(CAST(\"client_lat\", 'DOUBLE'))) * sin(toRadians(\"latitude\"))) + ((cos(toRadians(CAST(\"client_lat\", 'DOUBLE'))) * cos(toRadians(\"latitude\"))) * cos((toRadians(\"longitude\") - toRadians(CAST(\"client_lon\", 'DOUBLE'))))))) * 6371)"
          }
        ]
      }
    },
    "ioConfig": {
      "topic": "adsb-json",
      "topicPattern": None,
      "inputFormat": {
        "type": "kafka",
        "headerFormat": {
          "type": "string",
          "encoding": "UTF-8"
        },
        "keyFormat": {
          "type": "regex",
          "pattern": "([\\s\\S]*)",
          "columns": [
            "line"
          ]
        },
        "valueFormat": {
          "type": "json",
          "keepNullColumns": False,
          "assumeNewlineDelimited": False,
          "useJsonNodeReader": False
        },
        "headerColumnPrefix": "kafka.header.",
        "keyColumnName": "kafka.key",
        "timestampColumnName": "kafka.timestamp",
        "topicColumnName": "kafka.topic"
      },
      "replicas": 1,
      "taskCount": 2,
      "taskDuration": "PT3600S",
      "consumerProperties": {
        "bootstrap.servers": f"{config["kafka"]["bootstrap_server"]}",
        "security.protocol": "SASL_SSL",
        "sasl.mechanism": "PLAIN",
        "sasl.jaas.config": f"org.apache.kafka.common.security.plain.PlainLoginModule  required username=\"{config["kafka"]["username"]}\" password=\"{config["kafka"]["password"]}\";"
      },
      "autoScalerConfig": None,
      "pollTimeout": 100,
      "startDelay": "PT5S",
      "period": "PT30S",
      "useEarliestOffset": True,
      "completionTimeout": "PT1800S",
      "lateMessageRejectionPeriod": None,
      "earlyMessageRejectionPeriod": None,
      "lateMessageRejectionStartDateTime": None,
      "configOverrides": None,
      "idleConfig": None,
      "stopTaskCount": 1,
      "stream": "adsb-json",
      "useEarliestSequenceNumber": True,
      "type": "kafka"
    },
    "tuningConfig": {
      "type": "kafka",
      "appendableIndexSpec": {
        "type": "onheap",
        "preserveExistingMetrics": False
      },
      "maxRowsInMemory": 150000,
      "maxBytesInMemory": 0,
      "skipBytesInMemoryOverheadCheck": False,
      "maxRowsPerSegment": 5000000,
      "maxTotalRows": None,
      "intermediatePersistPeriod": "PT10M",
      "maxPendingPersists": 0,
      "indexSpec": {
        "bitmap": {
          "type": "roaring"
        },
        "dimensionCompression": "lz4",
        "stringDictionaryEncoding": {
          "type": "utf8"
        },
        "metricCompression": "lz4",
        "longEncoding": "longs"
      },
      "indexSpecForIntermediatePersists": {
        "bitmap": {
          "type": "roaring"
        },
        "dimensionCompression": "lz4",
        "stringDictionaryEncoding": {
          "type": "utf8"
        },
        "metricCompression": "lz4",
        "longEncoding": "longs"
      },
      "reportParseExceptions": False,
      "handoffConditionTimeout": 900000,
      "resetOffsetAutomatically": False,
      "segmentWriteOutMediumFactory": None,
      "workerThreads": None,
      "chatRetries": 8,
      "httpTimeout": "PT10S",
      "shutdownTimeout": "PT80S",
      "offsetFetchPeriod": "PT30S",
      "intermediateHandoffPeriod": "P2147483647D",
      "logParseExceptions": False,
      "maxParseExceptions": 2147483647,
      "maxSavedParseExceptions": 0,
      "skipSequenceNumberAvailabilityCheck": False,
      "repartitionTransitionDuration": "PT120S"
    }
  },
  "context": None
}

In [ ]:
requests.post(f"{druid_host}/druid/indexer/v1/supervisor", json.dumps(ingestion_spec), headers=druid_headers)

In [ ]:
sql = '''
    SELECT
        "__time",
        "hex_ident_val",
        "client_id",
        "distance_to_aircraft",
        "altitude",
        "ground_speed",
        "latitude",
        "longitude",
        "vertical_rate"
    FROM "adsb-json"
    ORDER BY "__time" DESC
    LIMIT 5
'''
display.sql(sql)

In [ ]:
requests.get(f"{druid_host}/druid/indexer/v1/supervisor", headers=druid_headers).json()

In [ ]:
requests.get(f"{druid_host}/druid/indexer/v1/tasks/?datasource=adsb-json&state=running", headers=druid_headers).json()

In [ ]:
requests.get(f"{druid_host}/druid/coordinator/v1/datasources/adsb-json/segments", headers=druid_headers).json()